In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pystan
import pandas as pd
import networkx as nx

plt.style.use('ggplot')
%matplotlib inline

In [2]:
# load nodes and edges from file
nodes = pd.read_csv('./data/got-s6-nodes.csv')
edges = pd.read_csv('./data/got-s6-edges.csv')

# initialize networkx graph object and add nodes and edges
G = nx.Graph(name='Game Of Thrones S06')
G.add_nodes_from(nodes['Id'])
edge_dict = [(x['Target'],x['Source'], {'weight' : x['Weight']}) for (_,x) in edges.iterrows()]
G.add_edges_from(edge_dict)

In [3]:
# see http://edwardlib.org/tutorials/latent-space-models

model_code = """
data { 
    int<lower=2> N; // number of nodes
    int<lower=1> D; // dimension of latent space
    int<lower=0> X[N,N]; // adjacency matrix
} parameters { 
    matrix[N,D] z; // latent positions in space
    real<lower=0> sigma; // std
} model {
    sigma ~ cauchy(0,10);
    
    for (n in 1:N) z[n,:] ~ normal(0, sigma);
    for (i in 1:N){
        for (j in 1:N) {
            if (i == j) continue; // This should probably be improved
            X[i,j] ~ poisson(1 / distance(z[i,:], z[j,:])); // can either be this or maybe poisson(exp(alpha-dist)) where alpha ~ normal(0, lambda) where lambda is small
        }
    }
    
}
"""


In [4]:
sm = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4162e9f4e72f08502a6b0a571fc8f675 NOW.
/home/david/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmpqgq8j2g7/stanfit4anon_model_4162e9f4e72f08502a6b0a571fc8f675_4052330872307066109.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [5]:
data = {'N' : len(G.nodes()), 'D' : 2, 'X' : nx.to_numpy_matrix(G, dtype=int)}

In [6]:
sm.sampling(data=data, iter=100)

Inference for Stan model: anon_model_4162e9f4e72f08502a6b0a571fc8f675.
4 chains, each with iter=100; warmup=50; thin=1; 
post-warmup draws per chain=50, total post-warmup draws=200.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
z[0,0]    -0.38    1.37   1.94  -3.09  -2.24  -0.27   1.49   2.38      2   7.86
z[1,0]    -2.83    1.66   2.35   -7.4  -4.61  -2.23  -0.69   0.02      2   4.55
z[2,0]    -0.64    0.68   0.96  -2.22  -1.52  -0.32   0.03   0.71      2   7.33
z[3,0]     5.54    0.55   0.95   4.32   4.72   5.18   6.36   7.43      3   3.38
z[4,0]     3.17    1.38   1.95  -0.44   1.39   3.91   4.24   6.05      2   6.16
z[5,0]    -0.67    3.39   4.79  -6.14  -5.24  -1.09   4.26   5.84      2  14.86
z[6,0]     1.02    1.22   1.73  -2.12  -0.34   1.65   2.31    2.9      2   9.24
z[7,0]    -0.29    1.07   1.85  -3.03  -2.29    0.7   1.39   1.86      3    2.9
z[8,0]     0.51    1.48    2.1  -3.67  -0.69   1.43    1.8   2.61      2  10.32
z[9,0]     1.18  